# Saget data parsing

In [31]:
from collections import Counter
import pandas as pd
import glob
import os
import json
def read_from_sagat(input_folder):
    # 创建一个字典来存储所有数据
    data_dict = {}

    # 获取当前目录下的所有Excel文件
    excel_files = glob.glob(f'{input_folder}/*.xlsx')

    # 迭代所有Excel文件
    for file in excel_files:
        # 获取文件名（不包含扩展名）
        file_name = os.path.splitext(os.path.basename(file))[0]

        # 读取当前Excel文件
        test_df = pd.read_excel(file)

        # 获取标答的正确次数
        standard_row = test_df.loc[test_df['你的姓名'] == '标答']
        person_total = standard_row['有关人所有问题答对次数'].values[0]
        car_total = standard_row['有关车所有问题答对次数'].values[0]
        other_total = standard_row['识别除人和车之外信息的正确次数'].values[0]

        # 迭代DataFrame中的每一行
        for index, row in test_df.iterrows():
            # 如果姓名是'标答'，则跳过该行
            if row['你的姓名'] == '标答':
                continue
            
            # 获取姓名和HUD类型
            name = row['你的姓名']
            hud_type = row['你使用的平视显示器为']

            # 如果名字不在字典中，创建一个新条目
            if name not in data_dict:
                data_dict[name] = {}

            # 如果HUD类型不在名字的字典中，创建一个新条目
            if hud_type not in data_dict[name]:
                data_dict[name][hud_type] = Counter({'src': []})

            # 计算正确率并更新Counter对象
            data_dict[name][hud_type]['person_correct_ratio'] += row['有关人所有问题答对次数'] / person_total
            data_dict[name][hud_type]['car_correct_ratio'] += row['有关车所有问题答对次数'] / car_total
            data_dict[name][hud_type]['other_correct_ratio'] += row['识别除人和车之外信息的正确次数'] / other_total

            # 添加数据来源
            data_dict[name][hud_type]['src'].append(file_name)

    # 在所有数据处理完成后遍历data_dict进行四舍五入
    for name in data_dict:
        for hud_type in data_dict[name]:
            data_dict[name][hud_type]['person_correct_ratio'] = round(data_dict[name][hud_type]['person_correct_ratio'], 2)
            data_dict[name][hud_type]['car_correct_ratio'] = round(data_dict[name][hud_type]['car_correct_ratio'], 2)
            data_dict[name][hud_type]['other_correct_ratio'] = round(data_dict[name][hud_type]['other_correct_ratio'], 2)


    # 导出以查看
    with open('input_2/batch2_data.json', 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

分开处理

In [44]:
from collections import Counter
import pandas as pd
import glob
import os
import json

def read_from_sagat(input_folder):
    # 创建一个字典来存储所有数据
    data_dict = {}

    # 获取当前目录下的所有Excel文件
    excel_files = glob.glob(f'{input_folder}/*.xlsx')

    # 迭代所有Excel文件
    for file in excel_files:
        # 获取文件名（不包含扩展名）
        file_name = os.path.splitext(os.path.basename(file))[0]

        # 读取当前Excel文件
        test_df = pd.read_excel(file)

        # 获取标答的正确次数
        standard_row = test_df.loc[test_df['你的姓名'] == '标答']
        person_total = standard_row['有关人所有问题答对次数'].values[0]
        car_total = standard_row['有关车所有问题答对次数'].values[0]
        other_total = standard_row['识别除人和车之外信息的正确次数'].values[0]

        # 迭代DataFrame中的每一行
        for index, row in test_df.iterrows():
            # 如果姓名是'标答'，则跳过该行
            if row['你的姓名'] == '标答':
                continue
            
            # 获取姓名和HUD类型
            name = row['你的姓名']
            hud_type = row['你使用的平视显示器为']

            # 如果名字不在字典中，创建一个新条目
            if name not in data_dict:
                data_dict[name] = {}

            # 如果HUD类型不在名字的字典中，创建一个新条目
            if hud_type not in data_dict[name]:
                data_dict[name][hud_type] = {}
            
            data_dict[name][hud_type][file_name] = {
                'person_correct_times': int(row['有关人所有问题答对次数']),
                'car_correct_times': int(row['有关车所有问题答对次数']),
                'other_correct_times': int(row['识别除人和车之外信息的正确次数']),
                'person_total': int(person_total),
                'car_total': int(car_total),
                'other_total': int(other_total),
            }

    # 导出以查看
    with open(f'{input_folder}/data_dict.json', 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False, indent=4)
read_from_sagat('input_2')

In [41]:
# 首先从JSON文件中读取data_dict
with open('input_2/data_dict.json', 'r', encoding='utf-8') as file:
    data_dict = json.load(file)

# 用于存储各个类别的总正确率和计数器的字典
avg_ratio_dict = {}

# 遍历data_dict来计算平均正确率
for name in data_dict:
    for hud_type in data_dict[name]:
        
        # 初始化总正确率和计数器
        total_person_ratio = 0
        total_all_ratio = 0
        total_car_ratio = 0
        total_other_ratio = 0
        count = 0
        
        for src in data_dict[name][hud_type]:
            
            # 累计各个类别的正确率
            total_person_ratio += data_dict[name][hud_type][src]['person_correct_ratio']
            total_car_ratio += data_dict[name][hud_type][src]['car_correct_ratio']
            total_other_ratio += data_dict[name][hud_type][src]['other_correct_ratio']
            total_all_ratio += data_dict[name][hud_type][src]['person_correct_ratio'] + \
                data_dict[name][hud_type][src]['other_correct_ratio'] + \
                data_dict[name][hud_type][src]['car_correct_ratio']

            count += 1
        
        # 计算平均正确率
        avg_person_ratio = total_person_ratio / count
        avg_car_ratio = total_car_ratio / count
        avg_other_ratio = total_other_ratio / count
        avg_all_ratio = total_all_ratio / count
        # 将平均正确率存储到新的键中
        data_dict[name][hud_type]['avg_ratio'] = {
            'avg_person_ratio': round(avg_person_ratio, 2),
            'avg_car_ratio': round(avg_car_ratio, 2),
            'avg_other_ratio': round(avg_other_ratio, 2),
            'avg_all_ratio': round(avg_all_ratio, 2)
        }

# 打印或保存更新后的data_dict
with open('input_2/batch2_data_updated.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)


In [65]:
import json

def calculate_avg_ratios(input_filepath, output_filepath):
    # 从JSON文件中读取data_dict
    with open(input_filepath, 'r', encoding='utf-8') as file:
        data_dict = json.load(file)

    # 遍历data_dict来计算平均正确率
    for name in data_dict:
        print(name)
        for hud_type in data_dict[name]:
            print(f"__{hud_type}")
   
            # 用于存储每个问卷的正确率
            ratios = {'person': [], 'car': [], 'other': [], 'all': []}
            #===video===#
            for src in data_dict[name][hud_type]:
                # 初始化变量来存储累计的正确次数和问题总数
                total_correct_times = {'person': 0, 'car': 0, 'other': 0, 'all': 0}
                total_questions = {'person': 0, 'car': 0, 'other': 0, 'all': 0}

                # 累加正确次数和问题总数
                for category in ['person', 'car', 'other']:
                    total_correct_times[category] = data_dict[name][hud_type][src][f'{category}_correct_times']
                    total_questions[category]     = data_dict[name][hud_type][src][f'{category}_total']

                    # 计算并存储每个问卷的正确率
                    ratios[category].append(data_dict[name][hud_type][src][f'{category}_correct_times'] / data_dict[name][hud_type][src][f'{category}_total'])

                # 计算和存储“all”类别的正确次数和问题总数
                total_correct_times['all'] = sum(data_dict[name][hud_type][src][f'{category}_correct_times'] for category in ['person', 'car', 'other'])
                total_questions['all']     = sum(data_dict[name][hud_type][src][f'{category}_total'] for category in ['person', 'car', 'other'])
                
                ratios['all'].append(total_correct_times['all'] / total_questions['all'])
            #===video===#
          
            # print(ratios)
            # 计算两种类型的平均正确率
            avg_ratio1 = {category: round(sum(ratios[category]) / len(ratios[category]),3) for category in ['person', 'car', 'other', 'all']}
            avg_ratio2 = {category: round(total_correct_times[category] / total_questions[category],3) for category in ['person', 'car', 'other', 'all']}

            # 将平均正确率存储到新的键中
            data_dict[name][hud_type]['avg_ratio1'] = avg_ratio1
            data_dict[name][hud_type]['avg_ratio2'] = avg_ratio2
    
    # 保存更新后的data_dict
    with open(output_filepath, 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

# 使用函数
calculate_avg_ratios('input_3/test_dict.json', 'input_3/test_output.json')


周儒
__不使用平视显示器
__单风险平视显示器
徐杨丽
__双风险平视显示器
__单风险平视显示器
__不使用平视显示器


map 的方式转到excel

In [30]:
import pandas as pd
import json

# 用JSON文件名替换'path/to/your/file.json'
with open('input_2/batch2_data.json', 'r', encoding='utf-8') as file:
    data_dict = json.load(file)
def dict_map_excel(segat_dict,input_folder):

    # HUD 类型的映射
    hud_mapping = {
        '不使用平视显示器': 'no_HUD',
        '单风险平视显示器': 'single_HUD',
        '双风险平视显示器': 'double_HUD'
    }

    # 创建一个列表来存储每行数据
    data_rows = []

    # 遍历data_dict来创建每行数据
    for name, hud_data in segat_dict.items():
        row_data = {'Name': name}

        for hud_type_cn, stats in hud_data.items():
            # 使用映射字典获取英文的 HUD 类型
            hud_type = hud_mapping[hud_type_cn]

            # 计算并添加各种正确率到行数据
            row_data[f'{hud_type}_all_1'] = stats['person_correct_ratio'] + stats['car_correct_ratio'] + stats['other_correct_ratio']
            row_data[f'{hud_type}_all_2'] = (stats['person_correct_ratio'] + stats['car_correct_ratio'] + stats['other_correct_ratio']) / 3
            row_data[f'{hud_type}_vehicle_1'] = stats['car_correct_ratio']
            row_data[f'{hud_type}_vehicle_2'] = stats['car_correct_ratio'] # 这里可能需要你提供更多信息来计算这个值
            row_data[f'{hud_type}_pedestrian_1'] = stats['person_correct_ratio']
            row_data[f'{hud_type}_pedestrian_2'] = stats['person_correct_ratio'] # 这里可能需要你提供更多信息来计算这个值
            row_data[f'{hud_type}_other_1'] = stats['other_correct_ratio']
            row_data[f'{hud_type}_other_2'] = stats['other_correct_ratio'] # 这里可能需要你提供更多信息来计算这个值

        # 添加行数据到data_rows
        data_rows.append(row_data)

    # 创建一个DataFrame
    df = pd.DataFrame(data_rows)

    # 保存DataFrame为Excel文件
    df.to_excel(f'sagat_spss/{input_folder}_output.xlsx', index=False)
dict_map_excel(data_dict,"input_2")


In [3]:
data_dict

{}

In [ ]:
import pandas as pd

# Step 1 & 2: 读取每一个Excel文件
# (请替换 'file1.xlsx', 'file2.xlsx', … 'file15.xlsx' 为您的文件名)
files = ['file1.xlsx', 'file2.xlsx', ..., 'file15.xlsx']
dfs = [pd.read_excel(file) for file in files]

# Step 3: 合并所有的数据
merged_df = pd.concat(dfs, ignore_index=True)

# Step 4: 进行初步的数据分析
# 查看数据的基本信息，包括每列的数据类型
merged_df.info()

# 查看描述性统计
merged_df.describe()

# 保存合并后的数据到一个新的Excel文件
merged_df.to_excel('merged_data.xlsx', index=False)

# 请在此处添加任何其他您想进行的分析
